In [1]:
import sympy as sy
from sympy import sin, cos, pi, sqrt
import math

def HTM(theta, a, d, alpha):
    return sy.Matrix([
        [cos(theta), -sin(theta), 0, a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
        [sin(theta)*sin(alpha), cos(theta)*sin(alpha), cos(alpha), d*cos(alpha)],
        [0, 0, 0, 1],
    ])

q = sy.Matrix(sy.MatrixSymbol('q', 7, 1))
d1, d3, d5, df, a4, a5, a7 = sy.symbols('d1, d3, d5, df, a4, a5, a7')


dh_params = (
    (q[0, 0], 0, d1, 0),
    (q[1, 0], 0, 0, -pi/2),
    (q[2, 0], 0, d3, pi/2),
    (q[3, 0], a4, 0, pi/2),
    (q[4, 0], a5, d5, -pi/2),
    (q[5, 0], 0, 0, pi/2),
    (q[6, 0], a7, 0, pi/2),
    (0, 0, df, 0)
)


Ts = [HTM(*dhparam) for dhparam in dh_params]

for i, T in enumerate(Ts):
    if i == 0:
        T_abs = [T]
    else:
        T_abs.append(T_abs[i-1] @ T)


os = [T[0:3, 3:4] for T in T_abs]
Rxs = [T[0:3, 0:1] for T in T_abs]
Rys = [T[0:3, 1:2] for T in T_abs]
Rzs = [T[0:3, 2:3] for T in T_abs]

Jos = [o.jacobian(q) for o in os]
JRxs = [r.jacobian(q) for r in Rxs]
JRys = [r.jacobian(q) for r in Rys]
JRzs = [r.jacobian(q) for r in Rzs]

t = sy.Symbol("t")
q1 = sy.Function("q1")
q2 = sy.Function("q2")
q3 = sy.Function("q3")
q4 = sy.Function("q4")
q5 = sy.Function("q5")
q6 = sy.Function("q6")
q7 = sy.Function("q7")

dq = sy.Matrix(sy.MatrixSymbol('dq', 7, 1))

T_abs_ = []
for T in T_abs:
    T_ = T.subs([
        (q[0,0], q1(t)),
        (q[1,0], q2(t)),
        (q[2,0], q3(t)),
        (q[3,0], q4(t)),
        (q[4,0], q5(t)),
        (q[5,0], q6(t)),
        (q[6,0], q7(t)),
    ])
    T_abs_.append(T_)


os_ = [T[0:3, 3:4] for T in T_abs_]
Rxs_ = [T[0:3, 0:1] for T in T_abs_]
Rys_ = [T[0:3, 1:2] for T in T_abs_]
Rzs_ = [T[0:3, 2:3] for T in T_abs_]

q_ = sy.Matrix([
    [q1(t)],
    [q2(t)],
    [q3(t)],
    [q4(t)],
    [q5(t)],
    [q6(t)],
    [q7(t)],
])
Jos_ = [o.jacobian(q_) for o in os_]
JRxs_ = [r.jacobian(q_) for r in Rxs_]
JRys_ = [r.jacobian(q_) for r in Rys_]
JRzs_ = [r.jacobian(q_) for r in Rzs_]

Jos_dot_ = [sy.diff(J, t) for J in Jos_]
JRxs_dot_ = [sy.diff(J, t) for J in JRxs_]
JRys_dot_ = [sy.diff(J, t) for J in JRys_]
JRzs_dot_ = [sy.diff(J, t) for J in JRzs_]


Jos_dot = []
JRxs_dot = []
JRys_dot = []
JRzs_dot = []
for Js, newJs in zip((Jos_dot_, JRxs_dot_, JRys_dot_, JRzs_dot_), (Jos_dot, JRxs_dot, JRys_dot, JRzs_dot)):
    for J in Js:
        newJs.append(J.subs([
        (sy.Derivative(q1(t),t), dq[0, 0]),
        (sy.Derivative(q2(t),t), dq[1, 0]),
        (sy.Derivative(q3(t),t), dq[2, 0]),
        (sy.Derivative(q4(t),t), dq[3, 0]),
        (sy.Derivative(q5(t),t), dq[4, 0]),
        (sy.Derivative(q6(t),t), dq[5, 0]),
        (sy.Derivative(q7(t),t), dq[6, 0]),
        (q1(t), q[0, 0]),
        (q2(t), q[1, 0]),
        (q3(t), q[2, 0]),
        (q4(t), q[3, 0]),
        (q5(t), q[4, 0]),
        (q6(t), q[5, 0]),
        (q7(t), q[6, 0]),
    ]))

os = [sy.expand(e) for e in os]
Rxs = [sy.expand(e) for e in Rxs]
Rys = [sy.expand(e) for e in Rys]
Rzs = [sy.expand(e) for e in Rzs]
Jos = [sy.expand(e) for e in Jos]
JRxs = [sy.expand(e) for e in JRxs]
JRys = [sy.expand(e) for e in JRys]
JRzs = [sy.expand(e) for e in JRzs]
Jos_dot = [sy.expand(e) for e in Jos_dot]
JRxs_dot = [sy.expand(e) for e in JRxs_dot]
JRys_dot = [sy.expand(e) for e in JRys_dot]
JRzs_dot = [sy.expand(e) for e in JRzs_dot]

expr_all = [os, Rxs, Rys, Rzs, Jos, JRxs, JRys, JRzs, Jos_dot, JRxs_dot, JRys_dot, JRzs_dot]
names = [str(i) for i in range(7)] + ["ee"]
expr_name = [
    ["o_" + n for n in names],
    ["rx_" + n for n in names],
    ["ry_" + n for n in names],
    ["rz_" + n for n in names],
    ["jo_" + n for n in names],
    ["jrx_" + n for n in names],
    ["jry_" + n for n in names],
    ["jrz_" + n for n in names],
    ["jo_" + n + "_dot" for n in names],
    ["jrx_" + n + "_dot" for n in names],
    ["jry_" + n + "_dot" for n in names],
    ["jrz_" + n + "_dot" for n in names],
]

In [4]:
from sympy.printing.numpy import NumPyPrinter

names = [str(i) for i in range(7)] + ["ee"]

common_w = "import numpy as np\nfrom math import cos as c\nfrom math import sin as s\nfrom math import tan as ta\nfrom math import sqrt as sq\n"

#numba_word_q = "@njit(\"f8[:, :](f8[:, :])\")\n"
#numba_word_q_dq = "@njit(\"f8[:, :](f8[:, :], f8[:, :])\")\n"

with open("src_py_/htm.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, os):
        numpy_word = "def o_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rxs):
        numpy_word = "def rx_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rys):
        numpy_word = "def ry_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rzs):
        numpy_word = "def rz_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/Jos.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, Jos):
        numpy_word = "def jo_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRxs.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRxs):
        numpy_word = "def jrx_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRys.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRys):
        numpy_word = "def jry_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRzs.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRzs):
        numpy_word = "def jrz_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/Jo_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, Jos_dot):
        numpy_word = "def jo_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

with open("src_py_/JRx_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRxs_dot):
        numpy_word = "def jrx_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

with open("src_py_/JRy_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRys):
        numpy_word = "def jry_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

with open("src_py_/JRz_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRzs):
        numpy_word = "def jrz_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")




In [5]:
def translate_hoge(original, done):
    with open(original, "r") as f, open(done, "w") as g:
        file_data = f.readlines()
        for line in file_data:
            line = line.replace('numpy', 'np').replace('1/2', '0.5').replace('(0.5)', '0.5')
            line = line.replace('np.cos', 'c').replace('np.sin', 's').replace('np.sqrt', 'sq')
            #line = line.replace('(q)', '(q, L1, L3, L4, L5_1, L5_2, L7, L8)')
            #line = line.replace('(q, dq)', '(q, dq, L1, L3, L4, L5_1, L5_2, L7, L8)')
            # line = line.replace(']])', ']], dtype=np.float64)')
            # line = line.replace('[0, 0, 0, 0, 0, 0, 0]', '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]')
            # line = line.replace('[0]', '[0.0]').replace(' 0]],', ' 0.0]],').replace('[1]', '[1.0]').replace('[[0,', '[[0.0,').replace('0.0, 0],', '0.0, 0.0],')
            line = line.replace('import np as np', 'import numpy as np')
            g.write(line)


translate_hoge("src_py_/htm.py", "src_py_no_class/htm.py")
translate_hoge("src_py_/Jos.py", "src_py_no_class/Jos.py")
translate_hoge("src_py_/JRxs.py", "src_py_no_class/JRxs.py")
translate_hoge("src_py_/JRys.py", "src_py_no_class/JRys.py")
translate_hoge("src_py_/JRzs.py", "src_py_no_class/JRzs.py")
translate_hoge("src_py_/Jo_dots.py", "src_py_no_class/Jo_dots.py")
translate_hoge("src_py_/JRx_dots.py", "src_py_no_class/JRx_dots.py")
translate_hoge("src_py_/JRy_dots.py", "src_py_no_class/JRy_dots.py")
translate_hoge("src_py_/JRz_dots.py", "src_py_no_class/JRz_dots.py")